# Clone my repo to get all paths

In [ ]:
# So Colab can use the python files...
!git clone --single-branch -b develop https://github.com/muellevin/Studienarbeit.git
%cd Studienarbeit/Detection_training

 and set all up

In [ ]:
import os
import Tensorflow.scripts.Paths as pp

CUSTOM_MODEL_NAME = 'raccoonModel_50ep_b16_efficientdetlit0_17070'
PRETRAINED_MODEL_NAME = 'ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/ssdlite_mobiledet_edgetpu_320x320_coco_2020_05_19.tar.gz'


paths = pp.paths
paths.setup_paths()

OUTPUT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'export')
SAVED_MODEL = os.path.join(OUTPUT_PATH, 'saved_model')
TFLITE_EXPORT = os.path.join(OUTPUT_PATH, 'tfliteexport')
CHECKPOINT_PATH = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME)

os.makedirs(CHECKPOINT_PATH, exist_ok=True)
os.makedirs(OUTPUT_PATH, exist_ok=True)

PIPELINE_CONFIG = os.path.join(paths.MODEL_PATH, CUSTOM_MODEL_NAME, 'pipeline.config')

# Setup for SSDLite MobileDet (TF1)

In [ ]:
!pip install -U numpy==1.19.5
!pip install -U pycocotools==2.0.1
!pip install tensorflow==1.15

import tensorflow as tf
assert tf.__version__.startswith('1')

In [ ]:
# building tensorflow
! pip install tf_slim

# Download TensorFlow Object detection library if not available
if not os.path.exists(os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection')):
    command = "git clone https://github.com/tensorflow/models {}".format(paths.APIMODEL_PATH)

    !{command}

# install TensorFlow
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths.PROTOC_PATH}
    !cd {paths.PROTOC_PATH} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths.PROTOC_PATH, 'bin'))
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

In [ ]:
# Verify it
VERIFICATION_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'builders', 'model_builder_test.py')
# Verify Installation
!python {VERIFICATION_SCRIPT}

In [ ]:
with open(pp.LABELMAP, 'w') as f:
    for label in pp.LABELS:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TRAINSET_PATH} -output_path {pp.TRAINSET_RECORD_PATH}
!python {pp.TF_RECORD_SCRIPT} -img_path {paths.TESTSET_PATH} -output_path {pp.TESTSET_RECORD_PATH}

In [ ]:
# Prepare for training
import tensorflow as tf
from google.protobuf import text_format
from object_detection.protos import pipeline_pb2
import os
pipeline = pipeline_pb2.TrainEvalPipelineConfig()

config_path = os.path.join(pp.APIMODEL_PATH, 'research', 'object_detection', 'samples', 'configs', 'ssdlite_mobiledet_edgetpu_320x320_coco_sync_4x4.config')
with tf.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline)

pipeline.train_input_reader.tf_record_input_reader.input_path[:] = [pp.TRAINSET_RECORD_PATH]
pipeline.train_input_reader.label_map_path = pp.LABELMAP
pipeline.eval_input_reader[0].tf_record_input_reader.input_path[:] = [pp.TESTSET_RECORD_PATH]
pipeline.eval_input_reader[0].label_map_path = pp.LABELMAP
pipeline.train_config.fine_tune_checkpoint = os.path.join(paths.PRETRAINED_MODEL_PATH, PRETRAINED_MODEL_NAME, 'fp32', 'model.ckpt')
pipeline.train_config.batch_size = 16
pipeline.train_config.num_steps = 50000
pipeline.model.ssd.num_classes = len(pp.LABELS)
# Enable ssdlite, this should already be enabled in the config we downloaded, but this is just to make sure.
pipeline.model.ssd.box_predictor.convolutional_box_predictor.kernel_size = 3
pipeline.model.ssd.box_predictor.convolutional_box_predictor.use_depthwise = True
pipeline.model.ssd.feature_extractor.use_depthwise = True
# Quantization Aware Training
pipeline.graph_rewriter.quantization.delay = 0
pipeline.graph_rewriter.quantization.weight_bits = 8
pipeline.graph_rewriter.quantization.activation_bits = 8

config_text = text_format.MessageToString(pipeline)                                                                                                                                                                                                        
with tf.gfile.Open(PIPELINE_CONFIG, "wb") as f:
    f.write(config_text)

In [ ]:
# Train it
from datetime import datetime
start = datetime.now()

TRAINING_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'model_main.py')
command = "python {} --model_dir={} --pipeline_config_path={}".format(TRAINING_SCRIPT, CHECKPOINT_PATH, PIPELINE_CONFIG)
print(command)
!{command}

end = datetime.now()
duration = end - start
seconds_in_hour = 60 * 60
hours, seconds = divmod(duration.seconds, seconds_in_hour)
minutes = int(seconds / 60)
print('TRAINING TIME:', str(hours) + ':' + str(minutes if minutes > 10 else '%02d' % minutes))

# Export it

In [ ]:
FREEZE_SCRIPT = os.path.join(paths.APIMODEL_PATH, 'research', 'object_detection', 'export_inference_graph.py ')
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_prefix={} --output_directory={} --add_postprocessing_op=true".format(FREEZE_SCRIPT , PIPELINE_CONFIG, os.path.join(CHECKPOINT_PATH, 'model.ckpt-50000'), OUTPUT_PATH)
!{command}

In [ ]:
# to tflite
# Convert to a tflite file (for CPU)
! tflite_convert \
  --output_file={os.path.join(TFLITE_EXPORT, CUSTOM_MODEL_NAME +'.tflite')} \
  --graph_def_file={os.path.join(OUTPUT_PATH, 'tflite_graph.pb')} \
  --inference_type=QUANTIZED_UINT8 \
  --input_arrays="normalized_input_image_tensor" \
  --output_arrays="TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3" \
  --mean_values=128 \
  --std_dev_values=128 \
  --input_shapes=1,320,320,3 \
  --allow_custom_ops

# Setup for EfficientDet-Lite

In [ ]:
!pip install -q tflite-model-maker
!pip install tensorflow==2.*

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# Upload images

In [ ]:
GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/Studienarbeit'

if os.name =='posix':
    from google.colab import drive

    drive.mount('/content/drive', force_remount=True)

import shutil
shutil.copy(os.path.join(GOOGLE_DRIVE_PATH, pp.DATASET_NAME), paths.IMAGE_PATH)

if os.path.exists(pp.DATASET):
  !tar -zxvf {pp.DATASET}

To lazy to do the  conversion again locally

In [ ]:
from Tensorflow.scripts.csv_to_voc import convert_csv_to_voc


convert_csv_to_voc(os.path.join(paths.TRAINSET_PATH, pp.CSV_FILE_NAME), paths.TRAINSET_PATH)
convert_csv_to_voc(os.path.join(paths.TESTSET_PATH, pp.CSV_FILE_NAME), paths.TESTSET_PATH)
convert_csv_to_voc(os.path.join(paths.DEVSET_PATH, pp.CSV_FILE_NAME), paths.DEVSET_PATH)

In [ ]:
label_map = {} # dictionary to store class names
for label in pp.LABELS:
    label_map[label['id']] = label['name']
# prepare data for training
train_data = object_detector.DataLoader.from_pascal_voc(
        paths.TRAINSET_PATH, paths.TRAINSET_PATH, label_map=label_map)
validation_data = object_detector.DataLoader.from_pascal_voc(
        paths.TESTSET_PATH, paths.TESTSET_PATH, label_map=label_map)
test_data = object_detector.DataLoader.from_pascal_voc(
        paths.DEVSET_PATH, paths.DEVSET_PATH, label_map=label_map)

In [ ]:
# choose model
spec = object_detector.EfficientDetLite0Spec()

In [ ]:
# train model
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=34, 
                               batch_size=16, 
                               train_whole_model=True)

In [ ]:
# evaluate
model.evaluate(test_data)

In [ ]:
# convert to tflite
model.export(export_dir=TFLITE_EXPORT, tflite_filename=CUSTOM_MODEL_NAME + '.tflite', label_filename='labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

# and evaluate this
model.evaluate_tflite(os.path.join(TFLITE_EXPORT, CUSTOM_MODEL_NAME + '.tflite'), test_data)

# Compile for TPU

In [ ]:
# Compile for TPU
# ! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

# ! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

# ! sudo apt-get update

# ! sudo apt-get install edgetpu-compiler

NUMBER_OF_TPUS =  1

!edgetpu_compiler {os.path.join(TFLITE_EXPORT, CUSTOM_MODEL_NAME + '.tflite')} -d --num_segments=$NUMBER_OF_TPUS

In [ ]:
# zip and download
!zip -r {os.path.join(GOOGLE_DRIVE_PATH, CUSTOM_MODEL_NAME + '.zip')} {TFLITE_EXPORT}